In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("votes.csv")
len(df)

73891

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73891 entries, 0 to 73890
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            73891 non-null  int64  
 1   PostId        73891 non-null  int64  
 2   VoteTypeId    73891 non-null  int64  
 3   UserId        22448 non-null  float64
 4   CreationDate  73891 non-null  object 
 5   BountyAmount  252 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 3.4+ MB


In [10]:
Posts = df.drop_duplicates('PostId')['PostId'].values
Posts = np.array(Posts)

In [63]:
burst = set()
for i in range(0, len(Posts)):
    postid = Posts[i]
    vote = df[(df['PostId']==postid) & (df['VoteTypeId']==2)]
    votemonth = vote[['Id', 'CreationDate']]
    votemonth['CreationDate'] = pd.to_datetime(votemonth['CreationDate'])
    votemonth = votemonth.set_index('CreationDate')
    votemonth = votemonth.resample('M').count().to_period('M')
    votemonth.rename(columns={'Id':'vote_count'},inplace = True)

    paras = [4, 8]
    for para in paras:
        votemonth['EMA'+str(para)] = pd.DataFrame.ewm(
            votemonth['vote_count'], span=para, ignore_na=True, adjust=False).mean()
    votemonth['MACD'] = votemonth['EMA4'] - votemonth['EMA8']
    votemonth['EMA5'] = pd.DataFrame.ewm(votemonth['MACD'], span=5, ignore_na=True, adjust=False).mean()
    votemonth['Signal'] = votemonth['EMA5'] * votemonth['MACD']
    votemonth['Histogram'] = votemonth['MACD']-votemonth['EMA5']

    cnt = 0
    for index, row in votemonth.iterrows():
        histogram = row['Histogram']
        macd = row['MACD']
        ema5 = row['EMA5']
        signal = row['Signal']
        if (macd>0) & (ema5>0):
            cnt += 1
        else:
            cnt = 0
        if cnt > 12:
            burst.add(postid)

print(len(burst))

D:\anaconda3\envs\Torch\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


14


In [65]:
postid = 4752626
vote = df[(df['PostId']==postid) & (df['VoteTypeId']==2)]
votemonth = vote[['Id', 'CreationDate']]
votemonth['CreationDate'] = pd.to_datetime(votemonth['CreationDate'])
votemonth = votemonth.set_index('CreationDate')
votemonth = votemonth.resample('M').count().to_period('M')
votemonth.rename(columns={'Id':'vote_count'},inplace = True)
votemonth.head()

D:\anaconda3\envs\Torch\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,vote_count
CreationDate,
2011-01,2
2011-02,0
2011-03,0
2011-04,0
2011-05,0


In [66]:
paras = [4, 8]
for para in paras:
    votemonth['EMA'+str(para)] = pd.DataFrame.ewm(
        votemonth['vote_count'], span=para, ignore_na=True, adjust=False).mean()
votemonth['MACD'] = votemonth['EMA4'] - votemonth['EMA8']
votemonth['EMA5'] = pd.DataFrame.ewm(votemonth['MACD'], span=5, ignore_na=True, adjust=False).mean()
votemonth['Signal'] = votemonth['EMA5'] * votemonth['MACD']
votemonth['Histogram'] = votemonth['MACD']-votemonth['EMA5']